# FAA Aircraft Registration Database

In [1]:
%load_ext lab_black

In [3]:
import pandas as pd
import requests
import matplotlib as mpl
import geopandas as gpd
import json
import numpy as np

In [4]:
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 1000
pd.options.display.max_colwidth = None

In [5]:
# https://www.faa.gov/licenses_certificates/aircraft_certification/aircraft_registry/releasable_aircraft_download/

In [6]:
# https://registry.faa.gov/database/ReleasableAircraft.zip

### Read master and reference files

In [8]:
master = pd.read_csv(
    "/Users/stiles/data/aircraft/ReleasableAircraft/MASTER.txt", low_memory=False
)

In [9]:
reference = pd.read_csv(
    "/Users/stiles/data/aircraft/ReleasableAircraft/ACFTREF.txt", low_memory=False
)

### Clean up columns

In [10]:
master.columns = (
    master.columns.str.strip()
    .str.lower()
    .str.replace(" ", "_", regex=False)
    .str.replace("(", "", regex=False)
    .str.replace(")", "", regex=False)
    .str.replace("-", "_", regex=False)
)

In [13]:
reference.columns = (
    reference.columns.str.strip()
    .str.lower()
    .str.replace(" ", "_", regex=False)
    .str.replace("(", "", regex=False)
    .str.replace(")", "", regex=False)
    .str.replace("-", "_", regex=False)
)

In [14]:
reference.loc[(reference["type_acft"] == "4"), "type"] = "Fixed wing single engine"
reference.loc[(reference["type_acft"] == "5"), "type"] = "Fixed wing multi engine"
reference.loc[(reference["type_acft"] == "6"), "type"] = "Rotorcraft"

In [15]:
master.name = master.name.str.strip()
master.certification = master.certification.str.strip()
master.type_registrant = master.type_registrant.str.strip()

### Owners

In [16]:
owners = (
    master.groupby(["type_registrant", "certification"])
    .agg("size")
    .reset_index(name="planes")
)

In [17]:
owners.sort_values("planes", ascending=False).head(10)

,type_registrant,certification,planes
30,1,1N,33481
592,7,1N,28352
21,1,1,27611
105,1,42,21016
261,3,1N,19148
33,1,1NU,13678
270,3,1T,12518
450,4,1N,8419
38,1,1U,8168
19,1,,7657


In [18]:
corptrans = owners[(owners["planes"] > 20) & (owners["certification"] == "1T")]

In [19]:
airlines = [
    "DELTA AIR LINES INC",
    "AMERICAN AIRLINES INC",
    "SKYWEST AIRLINES INC",
    "SWIFT AIR LLC",
    "UNITED AIRLINES INC",
    "SOUTHWEST AIRLINES CO",
    "JETBLUE AIRWAYS CORP",
    "HORIZON AIR INDUSTRIES",
    "ALASKA AIRLINES INC",
    "REPUBLIC AIRWAYS INC",
    "SPIRIT AIRLINES INC",
    "MESA AIRLINES INC",
]

In [20]:
airlines

['DELTA AIR LINES INC',
 'AMERICAN AIRLINES INC',
 'SKYWEST AIRLINES INC',
 'SWIFT AIR LLC',
 'UNITED AIRLINES INC',
 'SOUTHWEST AIRLINES CO',
 'JETBLUE AIRWAYS CORP',
 'HORIZON AIR INDUSTRIES',
 'ALASKA AIRLINES INC',
 'REPUBLIC AIRWAYS INC',
 'SPIRIT AIRLINES INC',
 'MESA AIRLINES INC']

In [21]:
cargo = [
    "UNITED PARCEL SERVICE CO",
    "FEDERAL EXPRESS CORP",
    "FEDERAL EXPRESS CORPORATION",
    "CARGO AIRCRAFT MANAGEMENT INC",
    "ATLAS AIR INC",
]

In [22]:
cargo

['UNITED PARCEL SERVICE CO',
 'FEDERAL EXPRESS CORP',
 'FEDERAL EXPRESS CORPORATION',
 'CARGO AIRCRAFT MANAGEMENT INC',
 'ATLAS AIR INC']

In [23]:
private = ["NETJETS SALES INC", "FLEXJET LLC"]

In [24]:
corptrans.sort_values(by="planes", ascending=False).head(10)

,type_registrant,certification,planes
270,3,1T,12518
599,7,1T,6986
457,4,1T,820
36,1,1T,239
539,5,1T,203
759,8,1T,97


In [25]:
airline_planes = master

In [26]:
airline_planes_slim = airline_planes[["name", "n_number"]]

In [27]:
airline_planes_slim.to_csv("output/airline_planes.csv", index=False)

### Airline planes

In [28]:
american = master[master["name"].str.contains("AMERICAN AIRLINES")]

In [29]:
american.iloc[583]

n_number                                                         783AN
serial_number                           30004                         
mfr_mdl_code                                                   1385213
eng_mfr_mdl                                                      54555
year_mfr                                                          2000
type_registrant                                                      3
name                                             AMERICAN AIRLINES INC
street                               1 SKYVIEW DR                     
street2                              MD 8B369                         
city                                                FORT WORTH        
state                                                               TX
zip_code                                                    76155     
region                                                               2
county                                                             439
countr

In [30]:
master["n_number"] = "N" + master["n_number"]

In [31]:
src = master.merge(reference, left_on="mfr_mdl_code", right_on="code")

In [32]:
src[src["n_number"] == "N302FD"]

,n_number,serial_number,mfr_mdl_code,eng_mfr_mdl,year_mfr,type_registrant,name,street,street2,city,state,zip_code,region,county,country,last_action_date,cert_issue_date,certification,type_aircraft,type_engine,status_code,mode_s_code,fract_owner,air_worth_date,other_names1,other_names2,other_names3,other_names4,other_names5,expiration_date,unique_id,kit_mfr,kit_model,mode_s_code_hex,unnamed:_34,code,mfr,model,type_acft,type_eng,ac_cat,build_cert_ind,no_eng,no_seats,ac_weight,speed,tc_data_sheet,tc_data_holder,unnamed:_13,type
206833,N302FD,41239,0260106,52174,2011,5,LOS ANGELES CITY,C/O LOS ANGELES FIRE DEPARTMENT,AIR OPS SECTION 16617 ARMINTA ST,VAN NUYS,CA,91406,4,037,US,20200915,20120207,1T,6,3,V,50623501,,20110122,,,,,,20240229,1063602,,,A32741,NaN,0260106,AGUSTA AEROSPACE CORP,AW139,6,3,1,0,2,17,CLASS 3,0,R00002RD,LEONARDO S P A,NaN,Rotorcraft


In [33]:
df = src.copy()

In [34]:
len(df[~df["name"].isin(airlines) | ~df["name"].isin(cargo)])

288410

In [35]:
planes = df[
    (~df["name"].isin(airlines))
    & (~df["name"].isin(cargo))
    & (df["type_aircraft"].str.contains("4|5"))
    & (df["certification"].str.contains("1T"))
]

In [36]:
len(planes)

14520

### Export

In [37]:
df.to_csv("/Users/stiles/data/aircraft/master_faa_owners_database.csv", index=False)